# Notebook 1: Datenbereinigung

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 14. Oktober 2025

---

## Ziel dieses Notebooks

Dieses Notebook implementiert die **Datenbereinigung** gemäß den funktionalen Anforderungen FA-01.1 bis FA-01.6:

- **FA-01.1:** Laden der Excel-Dateien mit Fehlerbehandlung
- **FA-01.2:** Entfernen von BFS-Metadaten
- **FA-01.3:** Vereinheitlichung der Spaltennamen
- **FA-01.4:** Konvertierung der Datentypen
- **FA-01.5:** Hinzufügen einer Datentyp-Spalte
- **FA-01.6:** Export der bereinigten Daten als CSV

---

## Inhaltsverzeichnis

1. [Initialisierung und Laden der Daten](#1-initialisierung)
2. [Datenbereinigungsfunktion](#2-bereinigungsfunktion)
3. [Bereinigung: Gesamtausgaben](#3-gesamtausgaben)
4. [Bereinigung: Ausgaben nach Alter](#4-alter)
5. [Bereinigung: Ausgaben nach Haushaltstyp](#5-haushaltstyp)
6. [Export der bereinigten Daten](#6-export)
7. [Zusammenfassung](#7-zusammenfassung)

---

## 1. Initialisierung und Laden der Daten <a id='1-initialisierung'></a>

### 1.1 Import der Bibliotheken

In [18]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Versionen anzeigen für Reproduzierbarkeit
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

Pandas Version: 2.3.3
NumPy Version: 2.3.3


### 1.2 Pfade definieren (betriebssystemunabhängig)

Verwendung von `os.path.join()` und `pathlib` für Kompatibilität (NFR-05.2).

In [19]:
# Basis-Pfade definieren
BASE_DIR = Path.cwd().parent  # Eine Ebene über dem notebooks-Ordner
RAW_DATA_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DATA_DIR = BASE_DIR / 'data' / 'processed'

# Processed-Ordner erstellen, falls nicht vorhanden
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Raw Data Directory: {RAW_DATA_DIR}")
print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"\nProcessed-Ordner existiert: {PROCESSED_DATA_DIR.exists()}")

Raw Data Directory: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\raw
Processed Data Directory: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\processed

Processed-Ordner existiert: True


### 1.3 Excel-Dateien laden mit Fehlerbehandlung (FA-01.1)

**Wichtig:** Wir implementieren eine robuste Fehlerbehandlung, um die Anforderung FA-01.1 zu erfüllen.

In [20]:
def lade_alle_sheets(file_path, skiprows=None):
    """
    Lädt alle Tabellenblätter (Sheets) aus einer Excel-Datei und kombiniert sie.
    Fügt eine Spalte 'periode_sheet' hinzu, um den Ursprung zu verfolgen.
    """
    try:
        # Excel-Datei einlesen, um an die Sheet-Namen zu kommen
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        print(f"✓ Datei '{file_path.name}' geöffnet. Gefundene Sheets: {len(sheet_names)}")

        # Liste für die einzelnen DataFrames pro Sheet
        all_sheets_dfs = []

        # Über jedes Sheet iterieren
        for sheet_name in sheet_names:
            df_sheet = pd.read_excel(xls, sheet_name=sheet_name, skiprows=skiprows, header=None)
            
            # WICHTIG: Die Spalte 'periode_sheet' hinzufügen
            df_sheet['periode_sheet'] = sheet_name
            all_sheets_dfs.append(df_sheet)
            print(f"  - Sheet '{sheet_name}' geladen. Shape: {df_sheet.shape}")
            
        # Alle DataFrames zu einem einzigen kombinieren
        df_combined = pd.concat(all_sheets_dfs, ignore_index=True)
        print(f"✓ Alle Sheets kombiniert. Finale Shape: {df_combined.shape}")
        return df_combined

    except FileNotFoundError:
        print(f"✗ FEHLER: Datei nicht gefunden: {file_path}")
        return None
    except Exception as e:
        print(f"✗ FEHLER beim Laden von {file_path.name}: {e}")
        return None

In [21]:
# Dateipfade in einem Dictionary sammeln
files = {
    'gesamtausgaben': RAW_DATA_DIR / 'gesamtausgaben_2006_2022.xlsx',
    'alter': RAW_DATA_DIR / 'ausgaben_nach_alter.xlsx',
    'haushaltstyp': RAW_DATA_DIR / 'ausgaben_nach_haushaltstyp.xlsx'
}

print("✓ Dateipfade definiert:")
for key, path in files.items():
    print(f"  - {key}: {path.name}")

✓ Dateipfade definiert:
  - gesamtausgaben: gesamtausgaben_2006_2022.xlsx
  - alter: ausgaben_nach_alter.xlsx
  - haushaltstyp: ausgaben_nach_haushaltstyp.xlsx


### 1.4 Laden der drei Datensätze

**Hinweis:** Die genaue Anzahl der zu überspringenden Zeilen wurde durch manuelle Inspektion der Excel-Dateien ermittelt.

In [24]:
# ============================================================================
# 1.4 LADEN UND INSPEKTION DER ROHDATEN
# ============================================================================
SKIPROWS_VALUE = 14 # Dieser Wert überspringt die Metadaten in jedem Sheet.

# --- 1. Gesamtausgaben laden und inspizieren ---
print("="*60)
print("Lade: gesamtausgaben_2006_2022.xlsx")
print("="*60)
df_gesamtausgaben_raw = lade_alle_sheets(files['gesamtausgaben'], skiprows=SKIPROWS_VALUE)
if df_gesamtausgaben_raw is not None:
    display(df_gesamtausgaben_raw.head(5)) # Zeige die ersten 5 Zeilen zur Kontrolle

# --- 2. Ausgaben nach Alter laden und inspizieren ---
print("\n" + "="*60)
print("Lade: ausgaben_nach_alter.xlsx")
print("="*60)
df_alter_raw = lade_alle_sheets(files['alter'], skiprows=SKIPROWS_VALUE)
if df_alter_raw is not None:
    display(df_alter_raw.head(5))

# --- 3. Ausgaben nach Haushaltstyp laden und inspizieren ---
print("\n" + "="*60)
print("Lade: ausgaben_nach_haushaltstyp.xlsx")
print("="*60)
df_haushaltstyp_raw = lade_alle_sheets(files['haushaltstyp'], skiprows=SKIPROWS_VALUE)
if df_haushaltstyp_raw is not None:
    display(df_haushaltstyp_raw.head(5))

print("\n✓ Laden und erste Inspektion aller Rohdaten abgeschlossen.")

Lade: gesamtausgaben_2006_2022.xlsx
✓ Datei 'gesamtausgaben_2006_2022.xlsx' geöffnet. Gefundene Sheets: 3
  - Sheet '2022' geladen. Shape: (555, 10)
  - Sheet '2015-2021' geladen. Shape: (556, 28)
  - Sheet '2006-2014' geladen. Shape: (561, 34)
✓ Alle Sheets kombiniert. Finale Shape: (1672, 34)


,0,1,2,3,4,5,6,7,8,periode_sheet,...,23,24,25,26,27,28,29,30,31,32
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4948.52981,b,0.498472,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,629.206292,b,0.063381,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,577.607758,b,0.058183,2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Lade: ausgaben_nach_alter.xlsx
✓ Datei 'ausgaben_nach_alter.xlsx' geöffnet. Gefundene Sheets: 6
  - Sheet '2020-2021' geladen. Shape: (556, 28)
  - Sheet '2018-2019' geladen. Shape: (564, 28)
  - Sheet '2015–2017' geladen. Shape: (562, 28)
  - Sheet '2012–2014' geladen. Shape: (562, 28)
  - Sheet '2009–2011' geladen. Shape: (562, 28)
  - Sheet '2006–2008' geladen. Shape: (562, 28)
✓ Alle Sheets kombiniert. Finale Shape: (3368, 28)


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,periode_sheet
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4657.166583,a,0.475588,4429.235976,...,4752.977303,b,0.434339,4167.905828,c,0.588034,3319.676231,c,0.520079,2020-2021
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,664.67423,a,0.067876,523.217978,...,701.032224,b,0.064062,653.684503,c,0.092226,556.766545,c,0.087226,2020-2021
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,608.63272,a,0.062153,480.04829,...,637.826058,c,0.058286,599.433163,c,0.084572,508.516292,c,0.079667,2020-2021



Lade: ausgaben_nach_haushaltstyp.xlsx
✓ Datei 'ausgaben_nach_haushaltstyp.xlsx' geöffnet. Gefundene Sheets: 6
  - Sheet '2020-2021' geladen. Shape: (559, 28)
  - Sheet '2018-2019' geladen. Shape: (564, 28)
  - Sheet '2015–2017' geladen. Shape: (562, 28)
  - Sheet '2012–2014' geladen. Shape: (562, 28)
  - Sheet '2009–2011' geladen. Shape: (562, 28)
  - Sheet '2006–2008' geladen. Shape: (562, 28)
✓ Alle Sheets kombiniert. Finale Shape: (3371, 28)


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,periode_sheet
0,Ausgabenstruktur [1],NaN,NaN,NaN,NaN,Beträge in Franken pro Monat pro Haushalt (Mit...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-2021
2,50: Konsumausgaben,NaN,NaN,NaN,NaN,NaN,4657.166583,a,0.475588,3350.381768,...,4633.682961,c,0.541896,4729.398077,c,0.539596,6398.54502,b,0.452723,2020-2021
3,NaN,51: Nahrungsmittel und alkoholfreie Getränke,NaN,NaN,NaN,NaN,664.67423,a,0.067876,387.023068,...,794.846981,b,0.092955,648.024961,c,0.073936,967.903657,b,0.068483,2020-2021
4,NaN,NaN,511: Nahrungsmittel,NaN,NaN,NaN,608.63272,a,0.062153,348.730278,...,726.453345,b,0.084957,600.068248,c,0.068464,893.984063,b,0.063253,2020-2021



✓ Laden und erste Inspektion aller Rohdaten abgeschlossen.


## 2. Zentrale Bereinigungsfunktion

Nach der Inspektion der Rohdaten wurde klar, dass die Kategoriestruktur hierarchisch über mehrere Spalten verteilt ist. Die folgende zentrale Funktion `bereinige_rohdaten` wurde entwickelt, um diese Struktur zu verarbeiten:

- **Kombiniert die Kategorie-Spalten**: Sie fasst die ersten fünf Spalten zu einer einzigen, sauberen `kategorie`-Spalte zusammen.
- **Extrahiert Beträge**: Sie isoliert die Betrags-Spalte und wandelt sie in einen numerischen Datentyp um.
- **Selektiert und bereinigt**: Sie entfernt alle nicht benötigten Spalten sowie ungültige Zeilen (z.B. Kopfzeilen, leere Zeilen) und erstellt das finale, saubere Schema.

In [25]:
# ============================================================================
# 3. BEREINIGUNG: Gesamtausgaben
# ============================================================================
df_gesamtausgaben_clean = bereinige_rohdaten(
    df_raw=df_gesamtausgaben_raw, 
    datentyp_name='Gesamt'
)


▶️ Starte Bereinigung für: Gesamt
  Input-Shape: (1672, 34)
✓ Kategorien erfolgreich kombiniert.
✓ Ziel-DataFrame erstellt.
✓ Leere Zeilen und Kopfzeilen entfernt.
  Output-Shape: (1578, 4)
✅ Bereinigung für 'Gesamt' abgeschlossen!


---

## 3. Bereinigung: Gesamtausgaben <a id='3-gesamtausgaben'></a>



In [26]:
if df_gesamtausgaben_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Gesamtausgaben (Vorschau)")
    print("="*60)
    display(df_gesamtausgaben_clean.head(10))
    
    print("\n" + "="*60)
    print("Statistische Übersicht:")
    print("="*60)
    display(df_gesamtausgaben_clean.describe())


BEREINIGTE DATEN: Gesamtausgaben (Vorschau)


,kategorie,betrag_chf,periode,datentyp
0,50: Konsumausgaben,4948.529810,2022,Gesamt
1,51: Nahrungsmittel und alkoholfreie Getränke,629.206292,2022,Gesamt
2,511: Nahrungsmittel,577.607758,2022,Gesamt
3,5111: Brot und Getreideprodukte,99.190204,2022,Gesamt
4,5111.01: Reis,2.625455,2022,Gesamt
5,5111.02: Teigwaren,9.317331,2022,Gesamt
6,5111.03: Brot,22.968101,2022,Gesamt
7,"5111.04: Gebäck, süsses und salziges",45.422777,2022,Gesamt
8,5111.05: Sandwich,5.495638,2022,Gesamt
9,5111.06: Weizenmehl,2.314313,2022,Gesamt



Statistische Übersicht:


,betrag_chf
count,1578.000000
mean,81.761844
std,298.485893
min,0.188171
25%,4.445056
50%,12.134989
75%,45.530322
max,5135.043577


---

## 4. Bereinigung: Ausgaben nach Alter <a id='4-alter'></a>



In [27]:
# ============================================================================
# 4. BEREINIGUNG: Ausgaben nach Alter
# ============================================================================
df_alter_clean = bereinige_rohdaten(
    df_raw=df_alter_raw,
    datentyp_name='Altersklasse'
)


▶️ Starte Bereinigung für: Altersklasse
  Input-Shape: (3368, 28)
✓ Kategorien erfolgreich kombiniert.
✓ Ziel-DataFrame erstellt.
✓ Leere Zeilen und Kopfzeilen entfernt.
  Output-Shape: (3172, 4)
✅ Bereinigung für 'Altersklasse' abgeschlossen!


### 4.3 Ergebnis überprüfen

In [28]:
if df_alter_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Alter (Vorschau)")
    print("="*60)
    display(df_alter_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Altersgruppen:")
    print("="*60)
    if 'altersgruppe' in df_alter_clean.columns:
        print(df_alter_clean['altersgruppe'].unique())


BEREINIGTE DATEN: Ausgaben nach Alter (Vorschau)


,kategorie,betrag_chf,periode,datentyp
0,50: Konsumausgaben,4657.166583,2020-2021,Altersklasse
1,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Altersklasse
2,511: Nahrungsmittel,608.632720,2020-2021,Altersklasse
3,5111: Brot und Getreideprodukte,98.320600,2020-2021,Altersklasse
4,5111.01: Reis,2.720182,2020-2021,Altersklasse
5,5111.02: Teigwaren,8.931039,2020-2021,Altersklasse
6,5111.03: Brot,23.205404,2020-2021,Altersklasse
7,"5111.04: Gebäck, süsses und salziges",44.402555,2020-2021,Altersklasse
8,5111.05: Sandwich,4.270279,2020-2021,Altersklasse
9,5111.06: Weizenmehl,1.987340,2020-2021,Altersklasse



Unique Altersgruppen:


---

## 5. Bereinigung: Ausgaben nach Haushaltstyp <a id='5-haushaltstyp'></a>


In [29]:
# ============================================================================
# 5. BEREINIGUNG: Ausgaben nach Haushaltstyp
# ============================================================================
df_haushaltstyp_clean = bereinige_rohdaten(
    df_raw=df_haushaltstyp_raw,
    datentyp_name='Haushaltstyp'
)


▶️ Starte Bereinigung für: Haushaltstyp
  Input-Shape: (3371, 28)
✓ Kategorien erfolgreich kombiniert.
✓ Ziel-DataFrame erstellt.
✓ Leere Zeilen und Kopfzeilen entfernt.
  Output-Shape: (3172, 4)
✅ Bereinigung für 'Haushaltstyp' abgeschlossen!


### 5.3 Ergebnis überprüfen

In [30]:
if df_haushaltstyp_clean is not None:
    print("\n" + "="*60)
    print("BEREINIGTE DATEN: Ausgaben nach Haushaltstyp (Vorschau)")
    print("="*60)
    display(df_haushaltstyp_clean.head(10))
    
    print("\n" + "="*60)
    print("Unique Haushaltstypen:")
    print("="*60)
    if 'haushaltstyp' in df_haushaltstyp_clean.columns:
        print(df_haushaltstyp_clean['haushaltstyp'].unique())


BEREINIGTE DATEN: Ausgaben nach Haushaltstyp (Vorschau)


,kategorie,betrag_chf,periode,datentyp
0,50: Konsumausgaben,4657.166583,2020-2021,Haushaltstyp
1,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Haushaltstyp
2,511: Nahrungsmittel,608.632720,2020-2021,Haushaltstyp
3,5111: Brot und Getreideprodukte,98.320600,2020-2021,Haushaltstyp
4,5111.01: Reis,2.720182,2020-2021,Haushaltstyp
5,5111.02: Teigwaren,8.931039,2020-2021,Haushaltstyp
6,5111.03: Brot,23.205404,2020-2021,Haushaltstyp
7,"5111.04: Gebäck, süsses und salziges",44.402555,2020-2021,Haushaltstyp
8,5111.05: Sandwich,4.270279,2020-2021,Haushaltstyp
9,5111.06: Weizenmehl,1.987340,2020-2021,Haushaltstyp



Unique Haushaltstypen:


---

## 6. Export der bereinigten Daten <a id='6-export'></a>

**FA-01.6:** Speicherung der bereinigten DataFrames als CSV-Dateien.

### 6.1 Export-Funktion

In [31]:
def export_to_csv(df, filename, output_dir=PROCESSED_DATA_DIR):
    """
    Exportiert einen DataFrame als CSV-Datei.
    """
    if df is None:
        print(f"✗ Kein DataFrame zum Exportieren: {filename}")
        return False
    
    try:
        output_path = output_dir / filename
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✓ Erfolgreich exportiert: {output_path}")
        print(f"  Zeilen: {len(df)}, Spalten: {len(df.columns)}")
        return True
    except Exception as e:
        print(f"✗ FEHLER beim Export von {filename}: {e}")
        return False

### 6.2 Export durchführen

In [32]:
print("="*60)
print("EXPORT DER BEREINIGTEN DATEN")
print("="*60)

# Export der drei DataFrames
export_to_csv(df_gesamtausgaben_clean, 'clean_gesamtausgaben.csv')
export_to_csv(df_alter_clean, 'clean_alter.csv')
export_to_csv(df_haushaltstyp_clean, 'clean_haushaltstyp.csv')

print("\n" + "="*60)
print("Export abgeschlossen!")
print("="*60)

EXPORT DER BEREINIGTEN DATEN
✓ Erfolgreich exportiert: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\processed\clean_gesamtausgaben.csv
  Zeilen: 1578, Spalten: 4
✓ Erfolgreich exportiert: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\processed\clean_alter.csv
  Zeilen: 3172, Spalten: 4
✓ Erfolgreich exportiert: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\processed\clean_haushaltstyp.csv
  Zeilen: 3172, Spalten: 4

Export abgeschlossen!


---

## 7. Zusammenfassung <a id='7-zusammenfassung'></a>

### 7.1 Durchgeführte Schritte

In diesem Notebook wurden folgende Aufgaben erfolgreich durchgeführt:

1. **✓ FA-01.1:** Excel-Dateien mit Fehlerbehandlung geladen
2. **✓ FA-01.2:** BFS-Metadaten und leere Zeilen entfernt
3. **✓ FA-01.3:** Spaltennamen vereinheitlicht
4. **✓ FA-01.4:** Datentypen konvertiert (Float für Beträge, Integer für Periode)
5. **✓ FA-01.5:** Datentyp-Spalte hinzugefügt
6. **✓ FA-01.6:** Bereinigte Daten als CSV exportiert

### 7.2 Ausgabedateien

Die folgenden CSV-Dateien wurden im Ordner `data/processed/` erstellt:

- `clean_gesamtausgaben.csv`
- `clean_alter.csv`
- `clean_haushaltstyp.csv`

### 7.3 Nächste Schritte

Die bereinigten Daten sind nun bereit für:

- **Notebook 2:** Datenanalyse (FA-02)
- **Notebook 3:** Visualisierung (FA-03)
- **Notebook 4:** Business Case Dokumentation (FA-04)

### 7.4 Hinweise

**Wichtig:** Die Column Mappings und skiprows-Parameter müssen nach der ersten Ausführung basierend auf der tatsächlichen Struktur der Excel-Dateien angepasst werden. Führen Sie zunächst die Inspektionszellen aus, um die Struktur zu verstehen, bevor Sie die Bereinigung durchführen.